# Precipitaciones
## datos de precipitaciones en la red de estaciones de Madrid

In [164]:
%%html
    <!-- Google tag (gtag.js) --> 
    
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-BCBM9LRTT2">
    </script> 
    <script> 
      window.dataLayer = window.dataLayer || []; function gtag(){dataLayer.push(arguments);} gtag('js', new Date()); gtag('config', 'G-BCBM9LRTT2'); 
      </script>

In [147]:
import duckdb
import plotly.express as px

In [166]:
%%capture
conn = duckdb.connect('/home/jcaubin/duck_test.db') 

%load_ext sql
%sql conn --alias duckdb

%config SqlMagic.displaylimit = 5 #conf de jupysql limite de filas a mostrar

In [167]:
%%sql 

select *
from v_calair
limit 5

Running query in 'duckdb'

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H,VALOR,VALIDEZ,ESTACION_DESC,ALTITUD,PARAMETRO,FX_DATA,TS
28,079,102,81,28079102_81_98,2025,01,13,1,1.3,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO,2025-01-13 13:31:19.379000,2025-01-13 01:00:00
28,079,102,81,28079102_81_98,2025,01,13,2,2.0,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO,2025-01-13 13:31:19.379000,2025-01-13 02:00:00
28,079,102,81,28079102_81_98,2025,01,13,3,1.45,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO,2025-01-13 13:31:19.379000,2025-01-13 03:00:00
28,079,102,81,28079102_81_98,2025,01,13,4,0.47,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO,2025-01-13 13:31:19.379000,2025-01-13 04:00:00
28,079,102,81,28079102_81_98,2025,01,13,6,0.47,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO,2025-01-13 13:31:19.379000,2025-01-13 06:00:00


In [168]:
%%sql --save precip_total_estacion

pte2<<
select ano, mes, dia, make_date(ano, cast(mes as int), dia) fecha , estacion,estacion_desc, sum(valor) total
from v_CALAIR 
where magnitud = 89 --precipitacion 
and validez = 'V'
--and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
and date_diff ('day',TS, current_date()) < (30)
group by all
order by all


Running query in 'duckdb'

In [151]:
%%sql 

select ano, mes, dia, avg(total) precip_media, max(total) precip_max, min(total) precip_min, median(total) precipt_mediana, stddev(total) desviacion, count(total) n
from precip_total_estacion
group by all
order by all

Generating CTE with stored snippets: 'precip_total_estacion'

Running query in 'duckdb'

ANO,MES,DIA,precip_media,precip_max,precip_min,precipt_mediana,desviacion,n
2025,02,6,0.0,0.0,0.0,0.0,0.0,5
2025,02,7,4.404999999999999,6.65,1.8000000000000003,4.45,1.7018698866573527,10
2025,02,8,0.018,0.1,0.0,0.0,0.03359894178227774,10
2025,02,9,0.0,0.0,0.0,0.0,0.0,10
2025,02,10,0.0,0.0,0.0,0.0,0.0,10


In [152]:
%%sql --save precip_total_estacion2

select * , row_number() over (PARTITION BY ano, mes, dia ORDER BY total, estacion) r, count() over (PARTITION BY ano, mes, dia ) c
from precip_total_estacion
order by ano, mes, dia, total, estacion

Generating CTE with stored snippets: 'precip_total_estacion'

Running query in 'duckdb'

ANO,MES,DIA,fecha,ESTACION,ESTACION_DESC,total,r,c
2025,02,6,2025-02-06,24,Casa de Campo,0.0,1,5
2025,02,6,2025-02-06,39,Barrio del Pilar,0.0,2,5
2025,02,6,2025-02-06,54,Ensanche de Vallecas,0.0,3,5
2025,02,6,2025-02-06,56,Plaza Elíptica,0.0,4,5
2025,02,6,2025-02-06,59,Juan Carlos I,0.0,5,5


In [153]:
%%sql 

select ano, mes, dia, avg(total) precip_media, max(total) precip_max, min(total) precip_min, median(total) precipt_mediana, stddev(total) desviacion, count(total) n
from precip_total_estacion2
where r > 1 and r < c
group by all
order by all

Generating CTE with stored snippets: 'precip_total_estacion2'

Running query in 'duckdb'

ANO,MES,DIA,precip_media,precip_max,precip_min,precipt_mediana,desviacion,n
2025,02,6,0.0,0.0,0.0,0.0,0.0,3
2025,02,7,4.449999999999999,6.3,2.75,4.45,1.4255324819669517,8
2025,02,8,0.01,0.05,0.0,0.0,0.019272482233188632,8
2025,02,9,0.0,0.0,0.0,0.0,0.0,8
2025,02,10,0.0,0.0,0.0,0.0,0.0,8


In [154]:
%%sql --save cuali

SELECT 
    ano, mes, dia, avg(total) over (PARTITION BY ano, mes, dia) precip_media, row_number() over (PARTITION BY ano, mes, dia ORDER BY total) r
FROM precip_total_estacion
QUALIFY 
   row_number() over (PARTITION BY ano, mes, dia ORDER BY total) >3 
   AND row_number() over (PARTITION BY ano, mes, dia ORDER BY total) < 7
order by ano, mes, dia, r
--GROUP BY ano, mes, dia;

Generating CTE with stored snippets: 'precip_total_estacion'

Running query in 'duckdb'

ANO,MES,DIA,precip_media,r
2025,02,6,0.0,4
2025,02,6,0.0,5
2025,02,7,4.404999999999999,4
2025,02,7,4.404999999999999,5
2025,02,7,4.404999999999999,6


In [155]:
%%sql

select distinct ano, mes, dia, precip_media
from cuali
order by ano, mes, dia

Generating CTE with stored snippets: 'cuali'

Running query in 'duckdb'

ANO,MES,DIA,precip_media
2025,02,6,0.0
2025,02,7,4.404999999999999
2025,02,8,0.018
2025,02,9,0.0
2025,02,10,0.0


In [156]:
%%sql 
pm<<
select  fecha 
,avg(total) precip_media, max(total) precip_max, min(total) precip_min, median(total) precipt_mediana
, stddev(total) desviacion, count(total) n
from precip_total_estacion
group by all
order by all


Generating CTE with stored snippets: 'precip_total_estacion'

Running query in 'duckdb'

In [157]:
#grafico de precipitaciones
import plotly.express as px

df = pm.DataFrame()
fig = px.bar(df, x="fecha", y="precip_media", title = 'Precipitación media')
fig.show()

In [158]:
#boxplot
df = pte2.DataFrame()

fig = px.box(df, x= 'fecha', y = 'total', title = 'Distribución precipitaciones', hover_name='ESTACION_DESC')
fig.show()


In [159]:
%sql --close duckdb